In [1]:
%pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=f07a5e2c135496431c54724218b4b075ce292ef0946869e66ead35d77d8f0457
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [2]:
import wget

In [3]:
# ## wget: url의 파일들을 가져올 수 있는 기능
wget.download('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')

'ratings_train.txt'

In [4]:
import pandas as pd

In [5]:
df_naver_movie_comment=pd.read_csv('./ratings_train.txt', sep='\t')
df_naver_movie_comment[:3] # id: 게시판 number, document: 댓글, label: 긍정1 부정 0

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


## 머신러닝 의한 감성 분석
- 0. label: 긍정 1, 부정 0
    - 1인것은 긍정 array로, 0인것은 부정 array로 합친다.
- 1. 형태소 분석을 통해 품사 분류를 진행
- 2. vector 토큰화 (단어를 숫자화(OHE이용?))
- 3. 목표 변수 생성
- 4. 머신러닝 학습
- 5. 머신러닝 학습을 통한 예측


### 0. 긍정 1, 부정 0
- label이 0에 해당하는 것과 1에 해당하는 것을 나눠서 합친다.?
- 일단 0 따로 1따로 나눠봐.
- 나눈 다음에 positive, negative로 나누고 그걸 positive + negative로 합친다.

In [6]:
condition_neg=(df_naver_movie_comment['label'] == 0 )
df_movie_negative=df_naver_movie_comment[condition_neg]

In [7]:
doc_negative=df_movie_negative['document'][:1000].values

In [8]:
condition_pos=(df_naver_movie_comment['label'] == 1 )
df_movie_positive=df_naver_movie_comment[condition_pos]

In [9]:
doc_positive=df_movie_positive['document'][:1000].values

In [53]:
type(doc_negative), type(doc_positive), len(doc_negative), len(doc_positive)

(numpy.ndarray, numpy.ndarray, 1000, 1000)

In [55]:
import numpy as np

In [56]:
doc_total = np.concatenate((doc_positive, doc_negative), axis=0)

In [57]:
len(doc_total), type(doc_total)

(2000, numpy.ndarray)

In [58]:
doc_total

array(['흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
       '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
       '액션이 없는데도 재미 있는 몇안되는 영화', ...,
       '우선 평점 1점의 이유는 현재 평점이 너무나 높기때문이다 간략해서 영화보는 내내 엄청나게 불편햇다 서울시장 후보아내 엄정화가 나와서 서른도넘은 섹시가수 몸에착달라붙고 가슴이거의다 보이는 의상을입고 춤을출때 불쾌함을넘어 상당히 안좋은기분을 느끼게해준다',
       '아정말 제대로실망스러운영화ㅡㅡ참 돈이아깝다...이딴걸왜보냐고',
       '원작이 너무 좋았다.. 일본은 영화 리메이크가 늘 한심하다..'], dtype=object)

### 형태소 분석을 통해 품사 분류를 진행


In [59]:
!pip install python-mecab-ko

In [60]:
from mecab import MeCab

In [61]:
mecab = MeCab()

문장 토큰화

In [62]:
tokenized_comment = list()

for comment in doc_total :
    comment_morphs=mecab.morphs(comment)
    # print(comment_morphs) # ., ,, 한글자 불용어 빼기 len(word)>1 #list로 되어있으므로 이를 문장으로 만든다.
    # mecab.nouns(comment_morphs) : 하지 않는 이유는 문장으로 진행해야하기 때문
    # 원래는 .pos로 품사를 확인한 뒤 조사, 기호는 제외해야함.
    # 문장을 list로 되어있다면 이걸 looping 돌려서 제외시켜야함. tagging으로 형태소를 looping해야함.
    # print(' '.join(comment_morphs))
    # .append(' '.join(comment_morphs))
    filtered_words = [w for w in comment_morphs if len(w) > 1]
    tokenized_comment.append(' '.join(filtered_words))


tokenized_comment

['.. 포스터 보고 초딩 영화 ... 오버 연기 조차 가볍 구나',
 '사이몬페그 익살 스런 연기 돋보였 영화 스파이더맨 에서 보이 커스틴 던스트 너무나 이뻐 보였',
 '액션 는데 재미 영화',
 '평점 건데 만한데 헐리우드 화려 너무 길들여져',
 '걍인피니트가짱이다 진짜',
 '마다 눈물나 90 향수 자극 허진호 감성 절제 멜로 달인',
 '담백 깔끔 해서 신문 기사 보다 자꾸 잊어버린다 사람 다는',
 '매번 긴장 ㅠㅠ',
 '사람 웃긴 바스코 이기면 락스 라고 바비 이기 아이돌 라고 깐다 그냥 어서 안달 처럼 보인다',
 '이건 정말 깨알 캐스팅 질퍽 산뜻 내용 구성 러진 깨알 !♥',
 '약탈자 위한 변명 이라 저놈 착한 절대 아닌걸요',
 '나름 심오 그냥 학생 선생 놀아나 영화 절대 아님',
 '면서 가능',
 '절대 평범 영화 아닌 수작 라는 말씀 드립니다',
 'kl 고추 버려야 할텐데',
 '',
 '센스 연출력 탁월 캐스팅 90 향수 그래서',
 '포스 위력 다시 깨닫 검사 연기 정말 어요 완전 명품 드라마',
 '재밌 는데 별점 이리 은고',
 '아직 드라마 내인생의 최고',
 '패션 대한 열정 안나 윈투어',
 '허허 .. 작가 정신 나간 유령 .. 재미있 네요',
 '영화 이렇게 평가 는지 모르',
 '단순 면서 은은 매력 영화',
 '어라 서리 굶주렸',
 '정말 어요 그래서 은데 방법 >.. ㅜㅡ',
 '윤제문 라는 멋진 배우 발견 어요 소소 일탈 잔잔 미소 머금 합니다 음악 조금 아쉽 네요 ㅠㅠ 은데 평점 올리 10 ^^',
 '데너 리스 타르 가르 .. 주인 .. 누이 근친상간 이나 다닐지 라도 소설 에선 제일 멋진 이메 라니 스터 는데 드라마 에선 드래곤 제일 멋지 웃음 감독 토르 다크 월드 잡수셨 을지라도 기본 선방',
 '영화 사람 영혼 어루만져 군요 거친 세상사 잠시 동화 영화 행복 네요',
 '이렇게 가슴 시리 드라마 을까 감동 자체',
 '재미있 어요',
 '아요',
 '너무 충격 기분 완전히 꺼지 느낌 .. 활력

In [63]:
mecab.nouns(tokenized_comment[2])

['액션', '재미', '영화']

In [64]:
mecab.pos(tokenized_comment[2])

[('액션', 'NNG'), ('는데', 'VV+EC'), ('재미', 'NNG'), ('영화', 'NNG')]

In [65]:
## 명사만 + 한글자 제외해버림.
nouns_list = list()
for word in tokenized_comment:
    words = mecab.pos(word)
    filtered_words = [w for w, pos in words if (pos == 'MAG' or pos == 'NNG') and len(w) > 1]
    nouns_list.append(' '.join(filtered_words))

In [66]:
# nouns_list = list()
# for word in doc_total :
#     words = mecab.morphs(word)
#     # 한글자 이상만 담기_chat에 물어봄
#     filtered_words = [w for w in words if len(w) > 1]
#     nouns_list.extend(filtered_words)
# nouns_list

In [67]:
len(nouns_list)

2000

In [68]:
type(nouns_list)

list

### 벡터 토큰화
- countVectorizer = CountVectorizer()

In [69]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [70]:
countVectorizer = CountVectorizer()

In [71]:
countVectorizer.fit(nouns_list)

CountVectorizer()

In [72]:
countVectorizer.vocabulary_

{'보고': 885,
 '초딩': 2121,
 '영화': 1517,
 '오버': 1548,
 '연기': 1487,
 '익살': 1748,
 '보이': 894,
 '너무나': 387,
 '액션': 1386,
 '재미': 1842,
 '평점': 2299,
 '화려': 2461,
 '너무': 386,
 '진짜': 2052,
 '마다': 617,
 '눈물': 412,
 '향수': 2407,
 '자극': 1786,
 '감성': 49,
 '절제': 1896,
 '달인': 457,
 '신문': 1283,
 '기사': 310,
 '보다': 887,
 '자꾸': 1788,
 '사람': 1014,
 '매번': 670,
 '긴장': 327,
 '그냥': 256,
 '어서': 1413,
 '안달': 1369,
 '정말': 1907,
 '깨알': 337,
 '캐스팅': 2183,
 '산뜻': 1038,
 '내용': 382,
 '구성': 227,
 '약탈자': 1395,
 '변명': 875,
 '절대': 1891,
 '학생': 2361,
 '선생': 1107,
 '가능': 4,
 '말씀': 654,
 '고추': 164,
 '센스': 1147,
 '연출력': 1499,
 '포스': 2306,
 '위력': 1657,
 '다시': 431,
 '검사': 106,
 '완전': 1571,
 '명품': 699,
 '드라마': 560,
 '별점': 884,
 '이리': 1720,
 '은고': 1691,
 '아직': 1358,
 '최고': 2130,
 '패션': 2270,
 '열정': 1503,
 '작가': 1814,
 '정신': 1913,
 '유령': 1664,
 '이렇게': 1717,
 '단순': 444,
 '매력': 668,
 '서리': 1093,
 '방법': 843,
 '배우': 854,
 '발견': 835,
 '소소': 1156,
 '일탈': 1778,
 '미소': 788,
 '음악': 1696,
 '조금': 1940,
 '올리': 1563,
 '주인': 1981,
 '누이': 411,
 '

In [73]:
tfidfVectorizer=TfidfVectorizer(use_idf=True)

In [74]:
tfidfVectorizer.fit(nouns_list)

TfidfVectorizer()

In [75]:
tfidfVectorizer.idf_

array([7.90825515, 7.90825515, 7.90825515, ..., 7.90825515, 7.50279005,
       7.90825515])

In [76]:
features = tfidfVectorizer.transform(nouns_list)

In [77]:
features.toarray() # 전체를 vector화

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### 목표변수생성

In [78]:
target = ['긍정'] * len(doc_positive) + ['부정'] * len(doc_negative)


In [79]:
from sklearn.svm import SVC

In [80]:
model = SVC()

In [81]:
len(features.toarray()), len(target)

(2000, 2000)

In [82]:
model.fit(features.toarray(), target)

SVC()

In [1]:
reviews_comment = ['추리물이 공포로만 빛을 발한다면.', '추리물보다는 공포물?그래도 나쁘지 않다!', '애거서 크리스티 소설“할로윈 파티” 원작 에르퀼 푸아로 시리즈의 이전 두작품과 달리 호러요소가 짙음', '브래너 형님 연기는 기깔난데.. 거 좀 잘찍어보쇼' ]

SyntaxError: unterminated string literal (detected at line 1) (104401591.py, line 1)

모델 교육이 끝나면 새로운 영화 리뷰 댓글들을 넣고 예측 하기